In [1]:
data_path='../../data/BX-CSV-Dump/'
itempath = data_path + 'EB-build-goods.sql'
receiptspath = data_path + '75000-out1.csv'

In [2]:
with open(receiptspath,"r") as receiptsfile:
    receiptsdata = receiptsfile.read().split('\n')

In [4]:
receiptsdata[1:10]

['2, 7, 11, 37, 45',
 '3, 3, 33, 42',
 '4, 5, 12, 17, 47',
 '5, 6, 18, 42',
 '6, 2, 4, 34',
 '7, 15, 16, 23, 40',
 '8, 2, 3, 29, 34',
 '9, 18, 23, 26, 35, 36',
 '10, 44, 45']

In [5]:
baskets = [line.split(", ")[1:] for line in receiptsdata[0:-1]]

In [6]:
baskets[0:5]

[['11', '21'],
 ['7', '11', '37', '45'],
 ['3', '33', '42'],
 ['5', '12', '17', '47'],
 ['6', '18', '42']]

In [9]:
with open(itempath,"r") as itemfile:
    lines = itemfile.read().split('\n')

In [11]:
lines[1:10]

["insert into goods values (1,'Lemon','Cake',8.95,'Food');",
 "insert into goods values (2,'Casino','Cake',15.95,'Food');",
 "insert into goods values (3,'Opera','Cake',15.95,'Food');",
 "insert into goods values (4,'Strawberry','Cake',11.95,'Food');",
 "insert into goods values (5,'Truffle','Cake',15.95,'Food');",
 "insert into goods values (6,'Chocolate','Eclair',3.25,'Food');",
 "insert into goods values (7,'Coffee','Eclair',3.5,'Food');",
 "insert into goods values (8,'Vanilla','Eclair',3.25,'Food');",
 "insert into goods values (9,'Napoleon','Cake',13.49,'Food');"]

In [12]:
items = [line.split("(")[1][0:-2].split(",") for line in lines[0:-1]]
items[0:5]

[['0', "'Chocolate'", "'Cake'", '8.95', "'Food'"],
 ['1', "'Lemon'", "'Cake'", '8.95', "'Food'"],
 ['2', "'Casino'", "'Cake'", '15.95', "'Food'"],
 ['3', "'Opera'", "'Cake'", '15.95', "'Food'"],
 ['4', "'Strawberry'", "'Cake'", '11.95', "'Food'"]]

In [13]:
itemMap = {line[0]:line[1]+" "+line[2] for line in items}

In [14]:
numItems = len(items)
numBaskets = len(baskets)

In [15]:
numItems

50

In [16]:
numBaskets

75000

In [17]:
def support(itemset):
    basketSubset = baskets
    for item in itemset: 
        basketSubset = [basket for basket in basketSubset if item in basket]
    return float(len(basketSubset))/float(numBaskets)

In [18]:
support(['2','24'])

0.00288

In [19]:
supportItems1 = []
minsupport=0.01
for item in range(numItems):
    itemset=[str(item)]
    if support(itemset)>=minsupport:
        supportItems1.append(str(item))

In [20]:
supportItems1

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49']

In [21]:
import itertools 

In [22]:
def aprioriIteration(i,supportItems,assocRules,newSupportItems,minsupport,minconfidence):
    
    for itemset in itertools.combinations(supportItems,i): 
        itemset = list(itemset)
        if support(itemset)>minsupport:
             for j in range(i):
                rule_to = itemset[j]
                rule_from = [x for x in itemset if x!=itemset[j]]
                confidence=support(itemset)/support(rule_from) 
                if confidence>minconfidence:
                    assocRules.append((rule_from,rule_to))
                    for x in itemset:
                        if x not in newSupportItems:
                            newSupportItems.append(x)
    return assocRules, newSupportItems

In [23]:
minsupport=0.01
minconfidence=0.5
assocRules=[]
newSupportItems=[]

assocRules, supportItems2 =  aprioriIteration(2,supportItems1,assocRules,newSupportItems,minsupport,minconfidence)

In [24]:
assocRules

[(['46'], '0'),
 (['0'], '46'),
 (['3'], '18'),
 (['3'], '35'),
 (['9'], '4'),
 (['5'], '22'),
 (['44'], '14'),
 (['32'], '16'),
 (['16'], '32'),
 (['35'], '18'),
 (['18'], '35'),
 (['28'], '27'),
 (['27'], '28'),
 (['33'], '42')]

In [25]:
assocRules, supportItems3 =  aprioriIteration(3,supportItems2,assocRules,newSupportItems,minsupport,minconfidence)

In [26]:
assocRules

[(['46'], '0'),
 (['0'], '46'),
 (['3'], '18'),
 (['3'], '35'),
 (['9'], '4'),
 (['5'], '22'),
 (['44'], '14'),
 (['32'], '16'),
 (['16'], '32'),
 (['35'], '18'),
 (['18'], '35'),
 (['28'], '27'),
 (['27'], '28'),
 (['33'], '42'),
 (['18', '35'], '3'),
 (['3', '35'], '18'),
 (['3', '18'], '35')]

In [27]:
def ruleMeta(rule):
    rule_from = [itemMap[x] for x in rule[0]]
    return rule_from,itemMap[rule[1]]

In [28]:
[ruleMeta(rule) for rule in assocRules]

[(["'Chocolate' 'Coffee'"], "'Chocolate' 'Cake'"),
 (["'Chocolate' 'Cake'"], "'Chocolate' 'Coffee'"),
 (["'Opera' 'Cake'"], "'Cherry' 'Tart'"),
 (["'Opera' 'Cake'"], "'Apricot' 'Danish'"),
 (["'Napoleon' 'Cake'"], "'Strawberry' 'Cake'"),
 (["'Truffle' 'Cake'"], "'Gongolais' 'Cookie'"),
 (["'Bottled' 'Water'"], "'Berry' 'Tart'"),
 (["'Apricot' 'Croissant'"], "'Blueberry' 'Tart'"),
 (["'Blueberry' 'Tart'"], "'Apricot' 'Croissant'"),
 (["'Apricot' 'Danish'"], "'Cherry' 'Tart'"),
 (["'Cherry' 'Tart'"], "'Apricot' 'Danish'"),
 (["'Tuile' 'Cookie'"], "'Marzipan' 'Cookie'"),
 (["'Marzipan' 'Cookie'"], "'Tuile' 'Cookie'"),
 (["'Cheese' 'Croissant'"], "'Orange' 'Juice'"),
 (["'Cherry' 'Tart'", "'Apricot' 'Danish'"], "'Opera' 'Cake'"),
 (["'Opera' 'Cake'", "'Apricot' 'Danish'"], "'Cherry' 'Tart'"),
 (["'Opera' 'Cake'", "'Cherry' 'Tart'"], "'Apricot' 'Danish'")]